# Submit Nanoribbon Workchain

In [ ]:
# General imports.
import numpy as np
import ipywidgets as ipw
from IPython.display import clear_output

%aiida
from aiida import engine, plugins
import aiidalab_widgets_base as awb

# Local imports.
from .nanoribbon.viewers import CdxmlUpload2GnrWidget
from .nanoribbon.editors  import NanoribbonReplicateEditor

# Work chains, data objects.
SinglefileData = plugins.DataFactory('singlefile')
StructureData = plugins.DataFactory('structure')
NanoribbonWorkChain = plugins.WorkflowFactory('nanotech_empa.nanoribbon')

In [ ]:
# Check if the SSSP_modified is installed, if not, install it

from aiida.orm import UpfFamily
from aiida.orm import QueryBuilder

query = QueryBuilder()
query.append(UpfFamily)

if 'SSSP_modified' not in [u.label for u in query.all(flat=True)]:
    print("First time setup: setting up UPF SSSP_modified.")
    ! wget -q https://polybox.ethz.ch/index.php/s/C4YaQ1zsmWXctk2/download -O $AIIDALAB_HOME/SSSP_modified.tar.gz
    ! tar xvf $AIIDALAB_HOME/SSSP_modified.tar.gz -C $AIIDALAB_HOME > /dev/null
    ! rm $AIIDALAB_HOME/SSSP_modified.tar.gz > /dev/null
    ! verdi data upf uploadfamily $AIIDALAB_HOME/SSSP_modified SSSP_modified "SSSP modified"

In [ ]:
rep_gnr = NanoribbonReplicateEditor(title='Supercell')
#dlink((empa_viewer, 'structure'), (build_slab, 'molecule'))
structure_selector = awb.StructureManagerWidget(
    importers=[
        awb.StructureUploadWidget(title="From computer"),
        awb.StructureBrowserWidget(title="AiiDA database"),
        awb.OptimadeQueryWidget(embedded=True),
        CdxmlUpload2GnrWidget(title="CDXML to GNR"),

    ],
    editors = [awb.BasicStructureEditor(title='Main'),rep_gnr],
    storable=False,
    node_class='StructureData')
display(structure_selector)

In [ ]:
style = {'description_width': '120px'}
layout = {'width': '70%'}

pw_code_dropdown = awb.CodeDropdown(input_plugin='quantumespresso.pw')
pp_code_dropdown = awb.CodeDropdown(input_plugin='quantumespresso.pp')
projwfc_code_dropdown = awb.CodeDropdown(input_plugin='quantumespresso.projwfc')

text_calc_description = ipw.Text(description='Calculation Name: ',
                          placeholder='A meaningful name',
                          style=style, layout=layout)
spin_u = ipw.Text(placeholder='1..10 15',
                                    description='IDs atoms spin U',
                                    style=style, layout={'width': '60%'})
spin_d = ipw.Text(placeholder='1..10 15',
                                    description='IDs atoms spin D',
                                    style=style, layout={'width': '60%'})
tot_charge = ipw.FloatText(
    value=0.0,
    step=0.1,
    description='Total charge:',
    disabled=False
)

cell_opt = ipw.Checkbox(
    value=True,
    description='Opt. cell',
    disabled=False,
    indent=False
)
max_nodes = ipw.IntText(
    value=60,
    description='Max # nodes',
    style={'description_width': 'initial'}, 
    disabled=False
)
mem_node = ipw.IntText(
    value=64,
    description='Max memory/node (GB)',
    style={'description_width': 'initial'}, 
    disabled=False
) 
display(ipw.VBox([cell_opt,spin_u, spin_d, tot_charge, text_calc_description, pw_code_dropdown, pp_code_dropdown, projwfc_code_dropdown, max_nodes, mem_node]))

In [ ]:
## spin of structure in strcture browser
def check_spins(atoms):
    tags = atoms.get_tags()
    su=np.where(tags == 1)[0].tolist()
    sd=np.where(tags == 2)[0].tolist()
    if len(su) == 0 and len(sd) == 0:
        add_name_old = '-no_spin'
    else:
        add_name_old = '-spin-' + str (abs(len(su) - len(sd)))
        
    su_str = awb.utils.list_to_string_range(su)
    sd_str = awb.utils.list_to_string_range(sd)

    return add_name_old, su_str, sd_str


@engine.calcfunction
def set_spins(NODE,U,D):
    
    su = U.value
    sd = D.value
    atoms=NODE.get_ase()
    tags = np.zeros(len(atoms))
    for u in  awb.utils.string_range_to_list(su)[0]:
        tags[u]=1
    for d in  awb.utils.string_range_to_list(sd)[0]:
        tags[d]=2    

    nu = len(awb.utils.string_range_to_list(su)[0])
    nd = len(awb.utils.string_range_to_list(sd)[0])
    if nu == 0 and nd  == 0:
        add_name = '-no_spin'
    else:
        if abs(nu - nd) > 0:
            add_name = '-spin-FM'
        else:
            add_name = '-spin-AM'
            
        
    atoms.set_tags(tags)

    new_struct = StructureData(ase=atoms)
    
    # ensure that tags got correctly translated into kinds 
    for t1, k in zip(tags, new_struct.get_site_kindnames()):
        t2 = int(k[-1]) if k[-1].isnumeric() else 0
        assert t1==t2
    
    #new_struct.store()
    old_name_split = NODE.description.split('spin')[0]
    if old_name_split == '':
        old_name_split = atoms.get_chemical_formula()    
    if old_name_split[-1] in ['_','-','.',',',' ']:
        old_name_split = old_name_split[:-1] 
    new_name = old_name_split + add_name    
    
    new_struct.description = new_name
    return new_struct

In [ ]:
## set initial spins according to tags of the structure
def on_structure_change(c=None):
    if structure_selector.structure:
        tags = structure_selector.structure.get_tags()
        su=np.where(tags == 1)[0].tolist()
        sd=np.where(tags == 2)[0].tolist()        
        su_str = awb.utils.list_to_string_range(su)
        sd_str = awb.utils.list_to_string_range(sd)
        spin_d.value=sd_str
        spin_u.value=su_str
        
structure_selector.observe(on_structure_change,names=['structure','input_structure'])

In [ ]:
def on_submit():    
    ## CHECK spin consistency and create new structure in case mismatch
    atoms = structure_selector.structure #ASE atoms
    description = structure_selector.structure_node.description
    if description == '':
        description = structure_selector.structure.get_chemical_formula()
    ## spin from strucure browser
    add_name_old, su_str, sd_str = check_spins(atoms)
    ## spin in input widgets
    input_su = awb.utils.list_to_string_range(awb.utils.string_range_to_list(spin_u.value)[0])
    input_sd = awb.utils.list_to_string_range(awb.utils.string_range_to_list(spin_d.value)[0])
    spin_mismatch = not ((input_su == su_str and input_sd == sd_str) or (input_sd == su_str and input_su == sd_str))    
        
    with submit_out:
        clear_output()
        if not structure_selector.structure_node:
            print("Please select a structure.")
            return
        if not pw_code_dropdown.selected_code:
            print("Please select a pw code.")
            return
        if not pp_code_dropdown.selected_code:
            print("Please select a pp code.")
            return
        if not projwfc_code_dropdown.selected_code:
            print("Please select a projwfc code.")
            return        
        if len(text_calc_description.value) < 3:
            print("Please enter a calculation name.")
            return
        
        ## IF input spins do not match selected structure spin, create new structure
        if spin_mismatch :
            node_to_submit = set_spins(structure_selector.structure_node,Str(input_su),Str(input_sd))
            old_name_split = description.split('spin')[0]
            old_name_split = old_name_split.split('no')[0] 
            if old_name_split[-1] in ['_','-','.',',',' ']:
                old_name_split = old_name_split[:-1] 

            print("created new structure to match spin guess. PK: ", 
                  node_to_submit.pk, ' with name: ' + node_to_submit.description )
            structure_selector.structure_node.description = old_name_split + add_name_old
            print("renamed input PK: ", structure_selector.structure_node.pk,  ' in: ' + old_name_split + add_name_old )
        else:
            node_to_submit = structure_selector.structure_node        

        builder = NanoribbonWorkChain.get_builder()
        try:
            builder.max_nodes = Int(max_nodes.value)
            builder.mem_node = Int(mem_node.value)
        except AttributeError:
            print("""The inputs `max_nodes` and `mem_node` will be ignored, consider updating the aiida-nanotech-empa  package to resolve the issue""")            
        builder.pw_code = pw_code_dropdown.selected_code
        builder.pp_code = pp_code_dropdown.selected_code
        builder.projwfc_code = projwfc_code_dropdown.selected_code
        if not cell_opt.value:
            builder.optimize_cell = Bool(False)
        builder.structure = node_to_submit
        builder.tot_charge = Float(tot_charge.value)
        builder.precision = Float(1.0)
        builder.pseudo_family = Str('SSSP_modified')
        builder.metadata = {
            "description": text_calc_description.value,
            "label": "NanoribbonWorkChain",
        }
        return builder

In [ ]:
btn_submit = awb.SubmitButtonWidget(NanoribbonWorkChain,input_dictionary_function=on_submit)
submit_out = ipw.Output()
display(btn_submit, submit_out)